In [ ]:
!ls

In [ ]:
!nvidia-smi

In [ ]:
!ls

In [1]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
config.allow_soft_placement=True
config.log_device_placement=True
sess = tf.Session(config=config)
#K.set_session(sess)


In [10]:
outputs_all = []

for i in range(2):
    with tf.device('/gpu:%d' % i):
        with tf.name_scope('tower_%d' % i):
            #outputs_all.append([i+1] * i)
            a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
            b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
            c = tf.matmul(a, b)
            print(c)
            outputs_all.append(c)
'''
merged = []
with tf.device('/cpu:0'):
    for op in outputs_all:
         tf.Concatenate(op)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess.run(c)
'''

print(c)

Tensor("tower_0_2/MatMul:0", shape=(2, 2), dtype=float32, device=/device:GPU:0)
Tensor("tower_1_2/MatMul:0", shape=(2, 2), dtype=float32, device=/device:GPU:1)
Tensor("tower_1_2/MatMul:0", shape=(2, 2), dtype=float32, device=/device:GPU:1)


In [9]:
outputs_all[1]

<tf.Tensor 'tower_1_1/MatMul:0' shape=(2, 2) dtype=float32>

In [ ]:
       for i in range(self.gpu_count):
            with tf.device('/gpu:%d' % i):
                with tf.name_scope('tower_%d' % i):
                    # Run a slice of inputs through this replica
                    zipped_inputs = zip(self.inner_model.input_names,
                                        self.inner_model.inputs)
                    inputs = [
                        KL.Lambda(lambda s: input_slices[name][i],
                                  output_shape=lambda s: (None,) + s[1:])(tensor)
                        for name, tensor in zipped_inputs]
                    # Create the model replica and get the outputs
                    outputs = self.inner_model(inputs)
                    if not isinstance(outputs, list):
                        outputs = [outputs]
                    # Save the outputs for merging back together later
                    for l, o in enumerate(outputs):
                        outputs_all[l].append(o)

        # Merge outputs on CPU
        with tf.device('/cpu:0'):
            merged = []
            for outputs, name in zip(outputs_all, output_names):


In [ ]:
import sys
import numpy as np
import tensorflow as tf
from datetime import datetime
 
shape=(int(10000),int(10000))
 
with tf.device("/gpu:0"):
    random_matrix = tf.random_uniform(shape=shape, minval=0, maxval=1)
    dot_operation = tf.matmul(random_matrix, tf.transpose(random_matrix))
    sum_operation = tf.reduce_sum(dot_operation)
 
startTime = datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
        result = session.run(sum_operation)
        print(result)
 
print("\n" * 2)
print("Time taken:", datetime.now() - startTime)
print("\n" * 2)

In [ ]:
!ls fashion/input

In [ ]:
import os 
import sys
import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
plt.rcParams["font.size"] = 15
import seaborn as sns
import json
from tqdm import tqdm
import pandas as pd
pd.set_option("display.max_rows", 101)
import glob
from collections import Counter
from PIL import Image

input_dir = "./fashion/input/"

In [ ]:
def classid2label(class_id):
    category, *attribute = class_id.split("_")
    return category, attribute

In [ ]:
def json2df(data):
    df = pd.DataFrame()
    for index, el in enumerate(data):
        for key, val in el.items():
            df.loc[index, key] = val
    return df

In [ ]:
!ls fashion/

In [ ]:
input_dir = "fashion/input/"
train_df = pd.read_csv(input_dir + "train.csv")
train_df.head()

In [ ]:
with open(input_dir + "label_descriptions.json") as f:
    label_description = json.load(f)
    
print("this dataset info")
print(json.dumps(label_description["info"], indent=2))

In [ ]:
category_df = json2df(label_description["categories"])
category_df["id"] = category_df["id"].astype(int)
category_df["level"] = category_df["level"].astype(int)
attribute_df = json2df(label_description["attributes"])
attribute_df["id"] = attribute_df["id"].astype(int)
attribute_df["level"] = attribute_df["level"].astype(int)

In [ ]:
print("Category Labels")
category_df

In [ ]:
print("Attribute Labels")
attribute_df

In [ ]:
sample_rle_encoding=train_df['EncodedPixels'][0]

In [ ]:
counter_category = Counter()
counter_attribute = Counter()
for class_id in train_df["ClassId"]:
    category, attribute = classid2label(class_id)
    counter_category.update([category])
    counter_attribute.update(attribute)

In [ ]:
category_name_dict = {}
for i in label_description["categories"]:
    category_name_dict[str(i["id"])] = i["name"]
attribute_name_dict = {}
for i in label_description["attributes"]:
    attribute_name_dict[str(i["id"])] = i["name"]

In [ ]:
ROOT_DIR="fashion/mrcnn"
DATA_DIR="fashion/input"

In [ ]:
sys.path.append(os.path.join(ROOT_DIR, 'Mask_RCNN'))

In [ ]:
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [ ]:
class FashionDataset(utils.Dataset):
    def load_fashion(self,image_ids=None,num_data=None):
        '''
        add_class --> register 46 classes self.add_class('fashion',i,name)
        image_ids --> unique index of images(name?)
        self.add_image('fashion',image_ids,width,height,annotations)
        width,height --> shape[:2] or extract form dataframe
        annotations --> all collections of annotations for each image
        Todo:
        There are some rows that have height and weight as nan value
        validation option is necessary for training
        '''
        for i,row in category_df.iterrows():
            self.add_class('fashion',i,row['name'])
        
        if image_ids is None:
            image_ids = list(set(train_df['ImageId']))
        
        if num_data is not None:
            random.seed(42)
            random.shuffle(image_ids)
            image_ids=image_ids[:num_data]
            
            
        for i in image_ids:
            Width = train_df[train_df['ImageId']==i]['Width'].reset_index(drop=True)[0]
            Height = train_df[train_df['ImageId']==i]['Height'].reset_index(drop=True)[0]
            self.add_image('fashion',
                           image_id=i,
                           path=DATA_DIR+'/train/'+i,
                           width=Width,
                           height=Height,
                           annotations=train_df[train_df['ImageId']==i])
        
        
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

    def load_image(self, image_id):
        info = self.image_info[image_id]
        ImagePath = info['path']
        image = np.asarray(Image.open(ImagePath).convert("RGB"))
        return image

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        annotations = info['annotations']
        width=info['width']
        height=info['height']
        
        instance_masks = []
        class_ids = []
        
        for i,annotation in annotations.iterrows():
            class_id=annotation['ClassId']
            class_id=class_id.split('_')[0]
            class_ids.append(class_id)
            rle = annotation['EncodedPixels']
            instance_masks.append(self.rle_to_mask(rle,width,height))
            
        if class_ids:
            mask = np.stack(instance_masks, axis=2).astype(np.bool)
            class_ids = np.array(class_ids, dtype=np.int32)+1
            return mask, class_ids
        else:
            # Call super class to return an empty mask
            return super(FashionDataset, self).load_mask(image_id)
        
        
    def rle_to_mask(self,rle,width,height):
        mask = np.zeros(width*height,dtype=np.int8)
        pixels_list = list(map(int,rle.split(" ")))
        for i in range(0,len(pixels_list),2):
            start_pixel = pixels_list[i]-1
            num_pixel = pixels_list[i+1]-1
            mask[start_pixel:start_pixel+num_pixel] = 1
        
        mask = mask.reshape((height,width),order='F')
        return mask
    
    def load_attributes(self, image_id):
        info = self.image_info[image_id]
        annotations = info['annotations']
        
        return annotations['ClassId'].apply(lambda x: x.split('_'))

In [ ]:
image_ids_list = list(set(train_df['ImageId']))

random.seed(42)
random.shuffle(image_ids_list)

val_split = 0.1
split = int((1-val_split)*len(image_ids_list))

train_ids = image_ids_list[:split]
val_ids = image_ids_list[split:]
train_ids = train_ids[:100]
val_ids = val_ids[:100]

In [ ]:
fashion_dataset = FashionDataset()
fashion_dataset.load_fashion(num_data=100)
fashion_dataset.prepare()

In [ ]:
fashion_dataset_train = FashionDataset()
fashion_dataset_train.load_fashion(train_ids)
fashion_dataset_val = FashionDataset()
fashion_dataset_val.load_fashion(val_ids)

In [ ]:
fashion_dataset_train.prepare()
fashion_dataset_val.prepare()

In [ ]:
print("Image Count: {}".format(len(fashion_dataset.image_ids)))
print("Class Count: {}".format(fashion_dataset.num_classes))
for i, info in enumerate(fashion_dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

In [ ]:
# Load and display random samples
image_ids = np.random.choice(fashion_dataset.image_ids, 4)
for image_id in image_ids:
    image = fashion_dataset.load_image(image_id)
    mask, class_ids = fashion_dataset.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, fashion_dataset.class_names)

In [ ]:
# Load random image and mask.
image_id = random.choice(fashion_dataset.image_ids)
image = fashion_dataset.load_image(image_id)
mask, class_ids = fashion_dataset.load_mask(image_id)
# Compute Bounding box
bbox = utils.extract_bboxes(mask)

# Display image and additional stats
print("image_id ", image_id, fashion_dataset.image_reference(image_id))
log("image", image)
log("mask", mask)
log("class_ids", class_ids)
log("bbox", bbox)
# Display image and instances
visualize.display_instances(image, bbox, mask, class_ids, fashion_dataset.class_names)

In [ ]:
# Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 1))
        run_lengths[-1] += 1
        prev = b
    return run_lengths
'''
def rle_to_mask(rle,width,height):
    mask = np.zeros(width*height,dtype=np.int8)
    pixels_list = list(map(int,rle.split(" ")))
    for i in range(0,len(pixels_list),2):
        start_pixel = pixels_list[i]-1
        num_pixel = pixels_list[i+1]-1
        mask[start_pixel:start_pixel+num_pixel] = 1
        
    mask = mask.reshape((height,width),order='F')
    return mask



m=rle_to_mask(sample_rle_encoding,3676,5214)
r=rle_encoding(m)

" ".join([str(e) for e in r]) == sample_rle_encoding
'''

In [ ]:
def extract_bboxes(mask):
    """Compute bounding boxes from masks.
    mask: [height, width, num_instances]. Mask pixels are either 1 or 0.
    Returns: bbox array [num_instances, (y1, x1, y2, x2)].
    """
    boxes = np.zeros([mask.shape[-1], 4], dtype=np.int32)
    for i in range(mask.shape[-1]):
        m = mask[:, :, i]
        # Bounding box.
        horizontal_indicies = np.where(np.any(m, axis=0))[0]
        vertical_indicies = np.where(np.any(m, axis=1))[0]
        if horizontal_indicies.shape[0]:
            x1, x2 = horizontal_indicies[[0, -1]]
            y1, y2 = vertical_indicies[[0, -1]]
            # x2 and y2 should not be part of the box. Increment by 1.
            x2 += 1
            y2 += 1
        else:
            # No mask for this instance. Might happen due to
            # resizing or cropping. Set bbox to zeros
            x1, x2, y1, y2 = 0, 0, 0, 0
        boxes[i] = np.array([y1, x1, y2, x2])
    return boxes.astype(np.int32)

In [ ]:
fashion_dataset.load_attributes(98)[np.array([False]*7)]

In [ ]:
masks,class_ids=fashion_dataset.load_mask(98)

In [ ]:
class_ids

In [ ]:
fashion_dataset.load_attributes(98)[class_ids<13].tolist()

In [ ]:
for i in range(1,10):
    print(i)

In [ ]:
masks[:,:,class_ids<13].shape

In [ ]:
bounding_boxes_test=[]
for id in fashion_dataset.image_ids:
    masks,class_ids=fashion_dataset.load_mask(id)
    masks = masks[:,:,class_ids<13]
    bb=extract_bboxes(masks)
    bounding_boxes_test.append(bb)

In [ ]:
s0=fashion_dataset.load_image(3)

In [ ]:
bounding_boxes_test

In [ ]:
tr0=[]
for bb in bounding_boxes_test[3]:
    tr0.append(s0[bb[0]:bb[2],bb[1]:bb[3]])

In [ ]:
len(tr0)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(tr0[1])

In [ ]:
long_class_id_df=train_df[train_df.ClassId.apply(lambda x: len(x.split('_')))>1]

In [ ]:
long_class_id_df.ClassId.apply(lambda x: x.split('_')[1:])

In [ ]:
def data_generator_phase2(dataset, shuffle=True,batch_size=1):
    b = 0  # batch item index
    image_index = -1
    image_ids = np.copy(dataset.image_ids)
    error_count = 0
    
    
    while True:
        try:
            # Increment index to pick next image. Shuffle if at the start of an epoch.
            image_index = (image_index + 1) % len(image_ids)
            if shuffle and image_index == 0:
                np.random.shuffle(image_ids)

            # Get GT bounding boxes and masks for image.
            image_id = image_ids[image_index]
            
            masks,class_ids = dataset.load_mask(image_id)
            attributes = dataset.load_attributes(image_id)[class_ids<13].tolist()#first element must be ignored
            
            masks = masks[:,:,class_ids<13]
            
            bbs = extract_bboxes(masks)
            
            original_img = dataset.load_image(image_id)

            for i,bb in enumerate(bbs):
                input_seg = original_img[bb[0]:bb[2],bb[1]:bb[3]]
                outputs = np.zeros((batch_size,92),dtype=np.int8)
                flag=False
                for i,index in enumerate(attributes[i]):
                    if i == 0:
                        continue
                    
                    outputs[b,int(index)]=1
                    flag=True
                
                if flag:
                    print(outputs)
                
                b += 1
                
                if b >= batch_size:
                    
                    yield input_seg,outputs
                    b=0
                    
                    
        except (GeneratorExit, KeyboardInterrupt):
            raise
        except:
            error_count += 1
            if error_count > 5:
                raise        

In [ ]:
phase2_gen=data_generator_phase2(fashion_dataset)

In [ ]:
for _ in range(0,100):
    inputs,outputs=next(phase2_gen)

In [ ]:
inputs

In [ ]:
outputs

In [ ]:
'''
how many examples under class id 12 have attributes?
All object --> 11540/77019 ~ 14%
picture 6696/45587 ~ 14%
Data augmentation?
-->multilabel classification

'''

'''
Todo: make dataset for attribute predictor

attribute_data_generator(dataset,~~~) // dataset --> Fashiondataset

fit_generator! class_id < 13! fit_generator(attribute_data_generator,~~~)
while True:
    img_index = (img_index+1) % len(img_ids)
    if img_index == 0:
        np.ramdom.shuffle(img_ids)
    img_id = img_ids[img_index]
    
    for object in img_id:

        ...
        
        yield input,output
        

Attribute --> long_class_id_df
encoded pixels --> bounding box
(width,height,channels),(class_num(92))
extract bounding box --> feed into model

'''
'''

def extract_bboxes(mask):
    """Compute bounding boxes from masks.
    mask: [height, width, num_instances]. Mask pixels are either 1 or 0.
    Returns: bbox array [num_instances, (y1, x1, y2, x2)].
    """
    boxes = np.zeros([mask.shape[-1], 4], dtype=np.int32)
    for i in range(mask.shape[-1]):
        m = mask[:, :, i]
        # Bounding box.
        horizontal_indicies = np.where(np.any(m, axis=0))[0]
        vertical_indicies = np.where(np.any(m, axis=1))[0]
        if horizontal_indicies.shape[0]:
            x1, x2 = horizontal_indicies[[0, -1]]
            y1, y2 = vertical_indicies[[0, -1]]
            # x2 and y2 should not be part of the box. Increment by 1.
            x2 += 1
            y2 += 1
        else:
            # No mask for this instance. Might happen due to
            # resizing or cropping. Set bbox to zeros
            x1, x2, y1, y2 = 0, 0, 0, 0
        boxes[i] = np.array([y1, x1, y2, x2])
    return boxes.astype(np.int32)

'''

In [ ]:
from keras.engine.topology import Layer
import keras.backend as K


class SpatialPyramidPooling(Layer):
    """Spatial pyramid pooling layer for 2D inputs.
    See Spatial Pyramid Pooling in Deep Convolutional Networks for Visual Recognition,
    K. He, X. Zhang, S. Ren, J. Sun
    # Arguments
        pool_list: list of int
            List of pooling regions to use. The length of the list is the number of pooling regions,
            each int in the list is the number of regions in that pool. For example [1,2,4] would be 3
            regions with 1, 2x2 and 4x4 max pools, so 21 outputs per feature map
    # Input shape
        4D tensor with shape:
        `(samples, channels, rows, cols)` if dim_ordering='th'
        or 4D tensor with shape:
        `(samples, rows, cols, channels)` if dim_ordering='tf'.
    # Output shape
        2D tensor with shape:
        `(samples, channels * sum([i * i for i in pool_list])`
    """

    def __init__(self, pool_list, **kwargs):

        self.dim_ordering = K.image_dim_ordering()
        assert self.dim_ordering in {'tf', 'th'}, 'dim_ordering must be in {tf, th}'

        self.pool_list = pool_list

        self.num_outputs_per_channel = sum([i * i for i in pool_list])

        super(SpatialPyramidPooling, self).__init__(**kwargs)

    def build(self, input_shape):
        if self.dim_ordering == 'th':
            self.nb_channels = input_shape[1]
        elif self.dim_ordering == 'tf':
            self.nb_channels = input_shape[3]

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.nb_channels * self.num_outputs_per_channel)

    def get_config(self):
        config = {'pool_list': self.pool_list}
        base_config = super(SpatialPyramidPooling, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def call(self, x, mask=None):

        input_shape = K.shape(x)

        if self.dim_ordering == 'th':
            num_rows = input_shape[2]
            num_cols = input_shape[3]
        elif self.dim_ordering == 'tf':
            num_rows = input_shape[1]
            num_cols = input_shape[2]

        row_length = [K.cast(num_rows, 'float32') / i for i in self.pool_list]
        col_length = [K.cast(num_cols, 'float32') / i for i in self.pool_list]

        outputs = []

        if self.dim_ordering == 'th':
            for pool_num, num_pool_regions in enumerate(self.pool_list):
                for jy in range(num_pool_regions):
                    for ix in range(num_pool_regions):
                        x1 = ix * col_length[pool_num]
                        x2 = ix * col_length[pool_num] + col_length[pool_num]
                        y1 = jy * row_length[pool_num]
                        y2 = jy * row_length[pool_num] + row_length[pool_num]

                        x1 = K.cast(K.round(x1), 'int32')
                        x2 = K.cast(K.round(x2), 'int32')
                        y1 = K.cast(K.round(y1), 'int32')
                        y2 = K.cast(K.round(y2), 'int32')
                        new_shape = [input_shape[0], input_shape[1],
                                     y2 - y1, x2 - x1]
                        x_crop = x[:, :, y1:y2, x1:x2]
                        xm = K.reshape(x_crop, new_shape)
                        pooled_val = K.max(xm, axis=(2, 3))
                        outputs.append(pooled_val)

        elif self.dim_ordering == 'tf':
            for pool_num, num_pool_regions in enumerate(self.pool_list):
                for jy in range(num_pool_regions):
                    for ix in range(num_pool_regions):
                        x1 = ix * col_length[pool_num]
                        x2 = ix * col_length[pool_num] + col_length[pool_num]
                        y1 = jy * row_length[pool_num]
                        y2 = jy * row_length[pool_num] + row_length[pool_num]

                        x1 = K.cast(K.round(x1), 'int32')
                        x2 = K.cast(K.round(x2), 'int32')
                        y1 = K.cast(K.round(y1), 'int32')
                        y2 = K.cast(K.round(y2), 'int32')

                        new_shape = [input_shape[0], y2 - y1,
                                     x2 - x1, input_shape[3]]

                        x_crop = x[:, y1:y2, x1:x2, :]
                        xm = K.reshape(x_crop, new_shape)
                        pooled_val = K.max(xm, axis=(1, 2))
                        outputs.append(pooled_val)

        if self.dim_ordering == 'th':
            outputs = K.concatenate(outputs)
        elif self.dim_ordering == 'tf':
            #outputs = K.concatenate(outputs,axis = 1)
            outputs = K.concatenate(outputs)
            #outputs = K.reshape(outputs,(len(self.pool_list),self.num_outputs_per_channel,input_shape[0],input_shape[1]))
            #outputs = K.permute_dimensions(outputs,(3,1,0,2))
            #outputs = K.reshape(outputs,(input_shape[0], self.num_outputs_per_channel * self.nb_channels))

        return outputs

In [ ]:
'''
import numpy as np
from keras.models import Sequential
from keras.layers import Convolution2D, Activation, MaxPooling2D, Dense
from spp.SpatialPyramidPooling import SpatialPyramidPooling

batch_size = #
num_channels = 3
num_classes = #

model = Sequential()

# uses theano ordering. Note that we leave the image size as None to allow multiple image sizes
model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=(3, None, None)))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(SpatialPyramidPooling([1, 2, 4]))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='sgd')

**Another choice could be tanh + hinge loss


model.fit(np.random.rand(batch_size, width, height, num_channels), np.zeros((batch_size, num_classes)))

'''

In [ ]:
class FashionConfig(Config):

    NAME = "fashion"

    IMAGES_PER_GPU = 1

    # Uncomment to train on 8 GPUs (default is 1)
    GPU_COUNT = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 46
    STEPS_PER_EPOCH=100
    
config = FashionConfig()
config.display()

In [ ]:
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=ROOT_DIR)

In [ ]:
model.log_dir

In [ ]:
'''
if args.model.lower() == "last":
        # Find last trained weights
        model_path = model.find_last()
elif args.model.lower() == "imagenet":
        # Start from ImageNet trained weights
        model_path = model.get_imagenet_weights()
else:
        model_path = args.model
'''
WEIGHT_PATH = 'last'

if WEIGHT_PATH == "last":
        # Find last trained weights
        model_path = model.find_last()
elif WEIGHT_PATH == "imagenet":
        # Start from ImageNet trained weights
        model_path = model.get_imagenet_weights()
else:
        pass
        #model_path = args.model

In [ ]:
model_path

In [ ]:
model.load_weights(model_path, by_name=True)

In [ ]:
epochs_stage1=40
epochs_stage2=120
epochs_stage3=160


# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(fashion_dataset_train, fashion_dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=epochs_stage1,
                layers='heads')

# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
print("Fine tune Resnet stage 4 and up")
model.train(fashion_dataset_train, fashion_dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=epochs_stage2,
            layers='4+')

# Training - Stage 3
# Fine tune all layers
print("Fine tune all layers")
model.train(fashion_dataset_train, fashion_dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=epochs_stage3,
            layers='all')

In [ ]:
class InferenceConfig(FashionConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
    
inference_config = InferenceConfig()
inference_config.display()

In [ ]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=ROOT_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
image_ids = train_ids[0]
image=fashion_dataset_train.load_image(0)

In [ ]:
result = model.detect([image])

In [ ]:
r = result[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            fashion_dataset_train.class_names)

In [ ]:
def get_test_filepaths(input_dir):
    jpg_fps = glob.glob(input_dir+'/'+'*.jpg')
    return list(set(jpg_fps))

test_input_dir=os.path.join(DATA_DIR, 'test')
test_fps=get_test_filepaths(test_input_dir)

In [ ]:
tf.__version__

In [ ]:
sys


In [ ]:
sys.path

In [ ]:
sys.executable